In [ ]:
# Python3

In [ ]:
# Uncomment the following cell if running as a Binder notebook, in order to hide the Jupyter header
# content and controls, so giving the appearance of a standalone application. Note that if you run
# this cell manually in a regular notebook session then the Jupyter header content and controls
# will still be hidden. Run this cell again to toggle back:

In [ ]:
%%javascript
$('#header').toggle()

In [ ]:
import ipywidgets as widgets
import re
import requests
from datetime import datetime
from ipyleaflet import FullScreenControl, LayerGroup, Map, Marker
from IPython.display import HTML
from urllib.parse import quote

In [ ]:
output_names = ['Description', 'Extract', 'Categories', 'Geo']

In [ ]:
# Layout styles
layout_main = {
    'width': '100%',
    'height': '100%',
}
layout_dropdown = {
    'width': '150px',
}
layout_button = {
    'width': '50px',
}

In [ ]:
# Image widgets
image_header = widgets.Image(
    value = open('images/Minipedia_Header.png', 'rb').read(),
    format = 'png',
    layout = layout_main,
)

In [ ]:
# Input widgets
input_text = widgets.Text(
    placeholder = 'Search for ...',
    layout = layout_main,
)

In [ ]:
# Control widgets
dropdown_set_mode = widgets.Dropdown(
    value = 0,
    options = [
        ('All outputs', 0),
        (output_names[0], 1),
        (output_names[1], 2),
        (output_names[2], 3),
        (output_names[3], 4),
    ],
    layout = layout_dropdown,
)
button_toggle_output_open = widgets.Button(
    icon = 'list',
    tooltip = 'Toggle all output displays',
    layout = layout_button,
)
button_clear = widgets.Button(
    icon = 'refresh',
    tooltip = 'Clear input and output',
    layout = layout_button,
)
button_go = widgets.Button(
    icon = 'play',
    tooltip = 'Go!',
    layout = layout_button,
)

In [ ]:
# Output widgets
output_message = widgets.Output()
output_0 = widgets.Output()
output_1 = widgets.Output()
output_2 = widgets.Output()
output_3 = widgets.Output()

In [ ]:
# Container widgets
children_header = [
    image_header,
    input_text,
]

children_control = [
    dropdown_set_mode,
    button_toggle_output_open,
    button_clear,
    button_go,
    output_message,
]

accordion_0 = widgets.Accordion(
    children = [output_0],
    layout = layout_main,
)
accordion_1 = widgets.Accordion(
    children = [output_1],
    layout = layout_main,
)
accordion_2 = widgets.Accordion(
    children = [output_2],
    layout = layout_main,
)
accordion_3 = widgets.Accordion(
    children = [output_3],
    layout = layout_main,
)
accordion_0.set_title(0, output_names[0])
accordion_1.set_title(0, output_names[1])
accordion_2.set_title(0, output_names[2])
accordion_3.set_title(0, output_names[3])

children_output = []

HBox_control = widgets.HBox(
    children = children_control,
)

VBox_main = widgets.VBox(
    layout = {
        'width': '90%',
        'min_height': '1500px',
        'left': '10%',
        'padding': '5px',
#         'border': '1px solid black',
    },
)

In [ ]:
@output_message.capture(clear_output=True, wait=True)
def set_output_message(html):
    display(HTML(html))

# -------------------------------------------------------------------------------------------------

@output_0.capture(clear_output=True, wait=True)
def set_output_0(html):
    accordion_0.selected_index = 0
    display(HTML(html))

# -------------------------------------------------------------------------------------------------

@output_1.capture(clear_output=True, wait=True)
def set_output_1(html):
    accordion_1.selected_index = 0
    display(HTML(html))

# -------------------------------------------------------------------------------------------------

@output_2.capture(clear_output=True, wait=True)
def set_output_2(html):
    accordion_2.selected_index = 0
    display(HTML(html))

# -------------------------------------------------------------------------------------------------

@output_3.capture(clear_output=True, wait=True)
def set_output_3(places):
    m = Map(
        center = (20.0, 0.0),
        zoom = 2,
    )
    markers = []
    for place in places.keys():
        if  ('lat' in places[place].keys()) \
        and ('lon' in places[place].keys()):
            markers.append(Marker(
                location = (places[place]['lat'], places[place]['lon']),
                title = place,
                draggable = False,
            ))
    m.add_layer(LayerGroup(
        layers = tuple(markers),
    ))
#     m.add_layer(MarkerCluster(
#         markers = tuple(markers),
#     ))
    m.add_control(FullScreenControl())
    accordion_3.selected_index = 0
    display(m)

In [ ]:
def set_output(dummy):
    global children_output
    children_output = []
    if (dropdown_set_mode.label == 'All outputs' or dropdown_set_mode.label == output_names[0]):
        children_output.append(accordion_0)
    if (dropdown_set_mode.label == 'All outputs' or dropdown_set_mode.label == output_names[1]):
        children_output.append(accordion_1)
    if (dropdown_set_mode.label == 'All outputs' or dropdown_set_mode.label == output_names[2]):
        children_output.append(accordion_2)
    if (dropdown_set_mode.label == 'All outputs' or dropdown_set_mode.label == output_names[3]):
        children_output.append(accordion_3)
    VBox_main.children = children_header + [HBox_control] + children_output

# -------------------------------------------------------------------------------------------------

def clear_output(dummy):
    for child in [accordion_0, accordion_1, accordion_2, accordion_3]:
        child.selected_index = None
    for output in [output_0, output_1, output_2, output_3]:
        output.clear_output()

# -------------------------------------------------------------------------------------------------

def clear_input_output(dummy):
    input_text.value = ''
    output_message.clear_output()
    clear_output(None)

# -------------------------------------------------------------------------------------------------

def toggle_control(dummy):
    control_disabled = children_control[0].disabled
    for child in children_control:
        child.disabled = not control_disabled

# -------------------------------------------------------------------------------------------------

def toggle_output_open(dummy):
    output_open = any([(child.selected_index == 0) for child in children_output])
    for child in children_output:
        child.selected_index = None if output_open else 0

# -------------------------------------------------------------------------------------------------

def go(dummy):
    if (not input_text.value):
        set_output_message('ERROR: Enter text for processing')
    else:

        toggle_control(None)
        set_output_message('Processing ...') # + '<i class="fa fa-gear fa-spin" style="font-size:18px"></i>')

        # -------------------------------------------------------------------------------------------------

        input_text_norm = input_text.value
        input_text_norm = input_text_norm[0].capitalize() + (input_text_norm[1:] if (len(input_text_norm) > 1) else '')
        input_text_norm = '_'.join(input_text_norm.split(' '))
        input_text_norm = input_text_norm.replace('|', '-')
        input_text_norm = input_text_norm.encode('utf-8')
        input_text_norm = quote(input_text_norm)

        # -------------------------------------------------------------------------------------------------

        url_modes = []

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == output_names[0])):
            url_modes.append('description')

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == output_names[1])):
            url_modes.append('extracts')

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == output_names[2])):
            url_modes.append('categories')

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == output_names[3])):
            url_modes.append('coordinates')

        url_modes = '|'.join(url_modes)

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == output_names[2])):
            url_modes += '&clshow=!hidden&cllimit=max'

        if ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == output_names[3])):
            url_modes += '&coprop=type|region|country'

        url = 'https://en.wikipedia.org'
        # url = 'https://simple.wikipedia.org'
        url += '/w/api.php'
        url += '?action=query&format=json&redirects&prop='
        url += url_modes
        url += '&titles='
        url += input_text_norm

        # -------------------------------------------------------------------------------------------------

        t1 = datetime.now()
        r = requests.get(url)
        t2 = datetime.now()

        deltat = t2 - t1
        deltat = round(deltat.seconds + (deltat.microseconds / 1e6), 6)

        # -------------------------------------------------------------------------------------------------

        toggle_control(None)
        clear_output(None)

        # -------------------------------------------------------------------------------------------------

        if  (r.ok) \
        and (r.json()) \
        and ('query' in r.json().keys()) \
        and ('pages' in r.json()['query'].keys()):

            set_output_message('Done in: ' + str(deltat) + ' s')

            pageid = list(r.json()['query']['pages'].keys())[0]
            title = r.json()['query']['pages'][pageid]['title']

# TODO: Use the following:
#             query_norm = None
#             if ('normalized' in r.json()['query'].keys()):
#                 query_norm = r.json()['query']['normalized'][0]['to']

#             query_redirect = None
#             if ('redirects' in r.json()['query'].keys()):
#                 query_redirect = r.json()['query']['redirects'][0]['to']

            if  ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == output_names[0])) \
            and ('description' in r.json()['query']['pages'][pageid].keys()):
                set_output_0(r.json()['query']['pages'][pageid]['description'])

            if  ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == output_names[1])) \
            and ('extract' in r.json()['query']['pages'][pageid].keys()):
                set_output_1(r.json()['query']['pages'][pageid]['extract'])

            if  ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == output_names[2])) \
            and ('categories' in r.json()['query']['pages'][pageid].keys()):
                set_output_2('<br/>'.join(
                    cat
                    for cat in [
                        re.search(r'Category:(.*)', cat['title'])[1]
                        for cat in r.json()['query']['pages'][pageid]['categories']
                    ]))

            if  ((dropdown_set_mode.label == 'All outputs') or (dropdown_set_mode.label == output_names[3])) \
            and ('coordinates' in r.json()['query']['pages'][pageid].keys()):

                set_output_3({title: {
                    'lat': r.json()['query']['pages'][pageid]['coordinates'][0]['lat'],
                    'lon': r.json()['query']['pages'][pageid]['coordinates'][0]['lon'],
                }})

# TODO: Use the following:
#                 geo_type = None
#                 if ('type' in r.json()['query']['pages'][pageid]['coordinates'][0].keys()):
#                     geo_type = r.json()['query']['pages'][pageid]['coordinates'][0]['type']

#                 geo_region = None
#                 if ('region' in r.json()['query']['pages'][pageid]['coordinates'][0].keys()):
#                     geo_region = r.json()['query']['pages'][pageid]['coordinates'][0]['region']

#                 geo_country = None
#                 if ('country' in r.json()['query']['pages'][pageid]['coordinates'][0].keys()):
#                     geo_country = r.json()['query']['pages'][pageid]['coordinates'][0]['country']

        else:
            set_output_message('ERROR: No data received')
            clear_output(None)

In [ ]:
dropdown_set_mode.observe(set_output)
button_toggle_output_open.on_click(toggle_output_open)
button_clear.on_click(clear_input_output)
button_go.on_click(go)

In [ ]:
set_output(None)
clear_output(None)
display(VBox_main)